In [16]:
import json
from tqdm import tqdm
# %%
import time
from tqdm import tqdm
import copy
import transformers
from vllm import LLM, SamplingParams
import json
import os
from datetime import datetime
from datasets import load_dataset
import random



/home/setup/miniconda3/envs/vllm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-09-21 15:29:36,725	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [18]:

pid = os.getpid()
seed = int(pid)+int(datetime.now().timestamp())
print("seed: ", seed)
random.seed(seed)


out_dir = "data"
os.system(f"mkdir -p {out_dir}")

current_time_no_symbols = datetime.now().strftime(
    "%Y-%m-%d %H:%M:%S").replace("-", "").replace(":", "").replace(" ", "")

seed:  1729505828


In [5]:
target_jsonl="source/split_twitter_archive_cleaned_aa.jsonl"
tweet_list=[]
with open(target_jsonl, "r") as f:
    for line in tqdm(f):
        tweet_list.append(json.loads(line))

10000000it [00:30, 328700.89it/s]


In [15]:
tweet_list[-11]

{'text': '美術の風景画出さなきゃいけないらしいのに捨てちゃった',
 'info': 'gs://hatakeyama_data_share/ver1/eiji_takahashi/archiveteam-twitter-stream-2022-04/twitter-stream-20220405.tar',
 'id': 1511233391974559748,
 'in_reply_to_status_id': None}

In [17]:

model_name = "cyberagent/calm3-22b-chat"
tensor_parallel_size = 1
# トークナイザーとモデルの準備
model = LLM(
    model=model_name,
    trust_remote_code=True,
    max_model_len=2000,
    tensor_parallel_size=tensor_parallel_size,
)


# %%
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)

# %%




INFO 09-21 15:29:48 llm_engine.py:223] Initializing an LLM engine (v0.6.1.post2) with config: model='cyberagent/calm3-22b-chat', speculative_config=None, tokenizer='cyberagent/calm3-22b-chat', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=2000, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=cyberagent/calm3-22b-chat, use_v2_block_manager=False, num_scheduler_steps=1, enable_prefix_caching=False, use_async_

Loading safetensors checkpoint shards:   0% Completed | 0/10 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  10% Completed | 1/10 [00:00<00:01,  6.33it/s]
Loading safetensors checkpoint shards:  20% Completed | 2/10 [00:00<00:03,  2.16it/s]
Loading safetensors checkpoint shards:  30% Completed | 3/10 [00:01<00:04,  1.68it/s]
Loading safetensors checkpoint shards:  40% Completed | 4/10 [00:02<00:03,  1.57it/s]
Loading safetensors checkpoint shards:  50% Completed | 5/10 [00:02<00:03,  1.52it/s]
Loading safetensors checkpoint shards:  60% Completed | 6/10 [00:03<00:02,  1.48it/s]
Loading safetensors checkpoint shards:  70% Completed | 7/10 [00:04<00:02,  1.45it/s]
Loading safetensors checkpoint shards:  80% Completed | 8/10 [00:05<00:01,  1.45it/s]
Loading safetensors checkpoint shards:  90% Completed | 9/10 [00:05<00:00,  1.46it/s]
Loading safetensors checkpoint shards: 100% Completed | 10/10 [00:06<00:00,  1.46it/s]
Loading safetensors checkpoint shards: 100% Completed | 10/10

INFO 09-21 15:29:59 model_runner.py:1008] Loading model weights took 41.9933 GB
INFO 09-21 15:30:00 gpu_executor.py:122] # GPU blocks: 1616, # CPU blocks: 227
INFO 09-21 15:30:02 model_runner.py:1311] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 09-21 15:30:02 model_runner.py:1315] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 09-21 15:30:34 model_runner.py:1430] Graph capturing finished in 32 secs.


In [66]:
"""
export CUDA_VISIBLE_DEVICES=0
export CUDA_VISIBLE_DEVICES=1
"""




def extract_random_text(ds):
    seed = int(pid)+int(datetime.now().timestamp())
    random.seed(seed)
    record = ds[random.randint(0, len(ds))]
    extract_length = random.randint(100, 300)
    try:
        extract_pos = random.randint(0, len(record["text"])-extract_length)
        extracted_text = record["text"][extract_pos:extract_pos +
                                        extract_length].strip()
    except:
        extracted_text = record["text"][0:100].strip()

    return extracted_text


# %%
# auto reload modules




# %%

max_tokens = 1024


def llm_gen(model, prompt_list,
            temperature=0.7, top_k=50,
            max_tokens=max_tokens,
            ):
    outputs = model.generate(
        prompt_list,
        sampling_params=SamplingParams(
            temperature=temperature,
            max_tokens=max_tokens,
            top_k=top_k,
            stop="\n\n", # 2回改行が出たら終了｡会話の二周目以降なので｡
        )
    )
    return [i.outputs[0].text.strip() for i in outputs]

def clean_output(text):
    if text.startswith("「") and text.endswith("」"):
        text = text[1:-1]
    return text


# %%
def ask_model(messages, model):
    messages = copy.deepcopy(messages)

    if messages[-1]["role"] == "user":
        messages.append({"role": "assistant", "content": ""})
    else:
        messages.append({"role": "user", "content": ""})

    prompt = tokenizer.apply_chat_template(messages, tokenize=False,)
    prompt = prompt.replace("<|im_end|>\n", "")
    text = llm_gen(model, [prompt], temperature=0.7,
                   top_k=50, max_tokens=1024)[0]
    return text

genres = [
    "心温まる日常会話",
    "楽しい会話",
    "ユーモラスな会話",
    "日常会話",
    "傾聴の会話",
    "相手の興味をそそる会話",
    "長めの会話",
    # "指示を聞いて答えるタイプの会話",
]




def gen_reply(tweet, genre):
    system_list = [
        f"あなたはSNSで{genre}をしています｡口調やノリを相手に合わせながら､返事をしてください｡返答のみを出力し、それ以外は何も出力しないでください｡",
    ]
    system_message = random.choice(system_list)


    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": tweet},
    ]
    text = ask_model(messages, model)
    text = clean_output(text)
    return text



In [68]:

def gen_prompt(tweet_list, genres):
    genre = random.choice(genres)
    tweet= random.choice(tweet_list)["text"]
    system_list = [
        f"あなたはSNSで{genre}をしています｡口調やノリを相手に合わせながら､返事をしてください｡返答のみを出力し、それ以外は何も出力しないでください｡",
    ]
    system_message = random.choice(system_list)

    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": tweet},
        {"role": "assistant", "content": ""},
    ]

    prompt = tokenizer.apply_chat_template(messages, tokenize=False,)
    prompt = prompt.replace("<|im_end|>\n", "")
    return prompt,tweet

In [69]:
batch_size=300


out_file_path = f"data/reply_{current_time_no_symbols}.jsonl"

with open(out_file_path, "a") as f:
    for i in tqdm(range(5*10**4)):
        try:
            prompts,tweets=zip(*[gen_prompt(tweet_list, genres) for i in range(batch_size)])
            replies= llm_gen(model, prompts, temperature=0.7,
                            top_k=50, max_tokens=256)
            replies=[clean_output(i) for i in replies]
            for prompt,tweet,reply in zip(prompts,tweets,replies):
                messages = [
                    #{"role": "system", "content": prompt},
                    {"role": "user", "content": tweet},
                    {"role": "assistant", "content": reply},
                ]
                f.write(json.dumps({"messages": messages}, ensure_ascii=False)+"\n")
        except Exception as e:
            print(e)
            time.sleep(5)
            continue
        time.sleep(3)












  0%|          | 0/50000 [00:04<?, ?it/s]
